In [1]:
# For loading the environment keys

from dotenv import load_dotenv
import os

load_dotenv()

True

In [2]:
from langfuse import Langfuse
from langchain_core.prompts import ChatPromptTemplate

# Creating the client that will connect with the project via keys and host
langfuse_client = Langfuse(
    secret_key = os.getenv("LANGFUSE_SECRET_KEY"),
    public_key = os.getenv("LANGFUSE_PUBLIC_KEY"),
    host = os.getenv("LANGFUSE_HOST")
)

In [5]:
langfuse_client.create_prompt(
    name = "HR-Assistant",
    type = "chat",
    prompt = [
        {
            "role" : "system",
            "content" : "You are {{experiencelevel}} HR"
        },
        {
            "role" : "user",
            "content" : "What should I do to take leave?"
        }
    ],
    labels = ["production", "hr"]
)

### Using Langchain to resolve the placeholders at runtime

In [10]:
from langchain_core.prompts import ChatPromptTemplate

# Use this for type="text" prompt
langfuse_prompt = langfuse_client.get_prompt("teacher-assistant", label="testing")
langchain_prompt = ChatPromptTemplate.from_template(langfuse_prompt.get_langchain_prompt())
langchain_prompt

ChatPromptTemplate(input_variables=['answer', 'criticlevel'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['answer', 'criticlevel'], input_types={}, partial_variables={}, template='As a {criticlevel} teaching expert, are you satisfied with the answer of {answer}? Explain in detail'), additional_kwargs={})])

In [11]:
# Use this for type="chat" prompt
langfuse_prompt = langfuse_client.get_prompt("HR-Assistant")
prompt_messages = langfuse_prompt.get_langchain_prompt()
langchain_prompt = ChatPromptTemplate.from_messages(prompt_messages)
langchain_prompt

ChatPromptTemplate(input_variables=['experiencelevel'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['experiencelevel'], input_types={}, partial_variables={}, template='You are {experiencelevel} HR'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='What should I do to take leave?'), additional_kwargs={})])

In [29]:
from langchain_ollama import ChatOllama

model = "llama3:8b"
temperature = 0.4
top_p = 0.1

llm = ChatOllama(model = model, temperature=temperature, top_p = top_p)

llm_chain = langchain_prompt | llm

user_input = {
    "experiencelevel" : "senior"
}

response = llm_chain.invoke(input = user_input)

In [30]:
print(response.content)

As a senior HR representative, I'd be happy to guide you through the process of taking leave.

To request leave, please follow these steps:

1. Check your company's leave policy: Review our company's leave policy to ensure you understand the types of leave available (e.g., vacation, sick, family and medical, etc.) and the procedures for requesting time off.
2. Determine the type of leave you need: Decide which type of leave you require (vacation, sick, or family and medical) and how many days you need to take off.
3. Submit your request: Fill out the Leave Request Form (available on our HR portal or by contacting me directly) with the following information:
	* Your name
	* The dates you wish to take leave
	* The type of leave requested
	* A brief explanation for the reason, if applicable (for family and medical leave only)
4. Submit your request in a timely manner: Try to submit your request at least 2 weeks prior to the start date of your leave, but no later than 1 week before.
5. Wai

### Using Python to resolve the placeholders at runtime

In [15]:
langfuse_prompt.config

{}

In [14]:
langfuse_prompt = langfuse_client.get_prompt("HR-Assistant")

compiled_prompt = langfuse_prompt.compile(experiencelevel = "fresher",
                                         chat_history = [
                                             {"role" : "user", "content" : "I am going to take a leave tomorrow"},
                                             {"role" : "user", "content" : "I want to encash my leaves"}
                                         ])

In [16]:
compiled_prompt

[{'role': 'system', 'content': 'You are fresher HR'},
 {'role': 'user', 'content': 'What should I do to take leave?'}]

In [17]:
import ollama

In [21]:
model = "llama3:8b"
temperature = 0.4
top_p = 0.1

response = ollama.chat(
    model = model,
    options={
        "temperature": temperature,  
        "top_p": top_p        
    },
    messages=compiled_prompt
)


In [28]:
print(response["message"]["content"])

To take a leave, you can follow these steps:

1. Check the company's leave policy: Before taking a leave, please review our company's leave policy to ensure you understand the rules and procedures.
2. Submit your request: You can submit your leave request through our HR system or by sending an email to me (your HR representative).
3. Provide necessary details: When submitting your request, please provide the following information:
	* Date(s) of leave
	* Reason for leave (optional)
	* Number of days off requested
4. Get approval: Once you've submitted your request, I'll review it and get back to you with an approval or rejection.
5. Confirm with your manager: If approved, please confirm the leave with your manager to ensure they're aware of your absence.

Some important notes:

* Leave requests should be made at least 2 days in advance (whenever possible).
* You can only take a maximum of [insert number] days off per year.
* If you need to take an extended leave, please discuss this wit